# COMP90014 Assignment 3
### Semester 2, 2023
Version 1.0 Last edited 20/09/2023.

In [ ]:
### Fill in your student details here
NAME = "Keziah Tikno"
ID = "1319716"

## Completing the assignment


***Networkx***

This assignment makes heavy use of the [networkx](https://networkx.org/documentation/stable/reference/introduction.html) graph package. <br>
You need to have `networkx >= 3.1` installed to complete the assignment. 

Clear instructions for how to use relevant networkx functionality will be given for each question. <br>

***Please ensure you use jupyter when modifying this document.***

For individual questions feel free to work on your solution in an IDE (i.e. VScode), but please only make modifications to this document while using jupyter. 


***Academic integrity***

This assignment should be completed by each student individually. <br>
Make sure you read this entire document, and ask for help if anything is not clear. <br>
Any changes or clarifications to this document will be announced via the LMS. <br>

Please make sure you review the University's rules on academic honesty and plagiarism: https://academichonesty.unimelb.edu.au/

Do not copy any code from other students or from the internet. This is considered plagiarism.

***Completing the assignment & submission***

To complete the assignment, finish the coding and short answer tasks in this notebook.<br>
Please do not copy or delete cells in this notebook, as this may interrupt the autograding of hidden tests. <br>
Your completed notebook file containing all your answers will be turned in via LMS. Please also submit an HTML file.

***Visible & hidden tests***

In some cases, we have provided test input and test output that you can use to try out your solutions. <br> 
These are visible tests and serve to warn you if you've made a mistake but are **not** exhaustive.

During assessment, there are also hidden tests we run to assess your code. <br>
As you won't see the hidden tests, it's up to you to decide whether your code is correct.

**Remember to save your work early and often.**

## Marking

***Graded cells***

Cells that must be completed to receive marks are clearly labelled with the following text:

`# -- GRADED CELL (N marks) - complete this cell --`

Only add answers to these cells. 

Some cells are code cells, in which you must complete the code to solve a problem. <br>
Others are markdown cells, in which you must write your answers to short-answer questions. 

<br>

***Completing code cells***

You will see the following text in graded code cells:

``` python
raise NotImplementedError()
```
You must remove the `raise NotImplementedError()` line from the cell, and replace it with your solution.<br>
If you want to import a library or use a helper function, the import statements must be **inside the solution cell** for that question. Ideally, it will be inside the function body. 

For example:
``` python
# -- GRADED CELL (2 marks) - complete this cell --

import numpy as np  # here

def your_function_name(args):
    """
    function description
    """
    import numpy as np  # or here
```

Include code comments in your solutions! <br>
Well commented code can help you to receive partial marks even if the final solution is incorrect. 

<br>

***Editing the notebook***

Only graded cells will be marked.
- Do **NOT** enter solutions outside of graded cells
- Do **NOT** duplicate or remove cells from the notebook
- You may add new cells to test code, but new cells will not be graded.
- Word limits, where stated, will be strictly enforced. Answers exceeding the limit **will not be marked**.

<br>

***Marks***

- The total marks for the assignment add up to 40.
- This assignment will be worth 15% of your overall subject grade.
- No marks are allocated to commenting in your code, except for where it helps you achieve partial marks (see above).
- We do however, encourage efficient and well commented code.

## Submission

Make sure you have filled in any place that says `# -- GRADED CELL (N marks) - complete this cell --`.

Before you turn this assignment in, make sure everything runs as expected, and the output is cleared. 

First, **restart the kernel**:

- In the menubar, select Kernel -> Restart.

Next, **run all cells**:

- In the menubar, select Cell -> Run All.

Finally, **clear all output**:

- In the menubar, select Options -> Clear All Outputs

Your completed notebook file containing all your answers must be turned in via LMS in `.ipynb` format. <br>
You must also submit a copy of this notebook in `html` format with the output cleared.

Your submission should include **only two** files with names formatted as: **Assignment_3.ipynb** and **Assignment_3.html**



<div style="background: rgb(255,165,0); border: solid 1px rgb(129,199,132); padding: 10px;">    

<h1>SETUP AND DATA</h1>

</div>

### Download A3 Data and Util Functions


In [ ]:
# if running on Linux or Mac
!rm -rf A3
!git clone -b A3 https://github.com/melbournebioinformatics/COMP90014 A3

In [ ]:
# if running on Windows
# !Remove-Item -Path A3 -Force -Recurse
# !git clone -b A3 https://github.com/melbournebioinformatics/COMP90014 A3

### Set Constants


In [ ]:
FASTQ1_PATH = 'A3/data/readset1.fq'
FASTQ2_PATH = 'A3/data/readset2.fq'
DBG1_FILEPATH = 'A3/data/dbg1.pkl'
DBG2_FILEPATH = 'A3/data/dbg2.pkl'
IST1_FILEPATH = 'A3/data/q4a_ist.pkl'
IST2_FILEPATH = 'A3/data/q4b_ist.pkl'

### Load Packages
For this assignment you may need to install some packages.

In [ ]:
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pickle
import pydot
import mmh3
from Bio import Align

from networkx.drawing.nx_agraph import graphviz_layout # If using this opt must have pygraphviz installed
#from networkx.drawing.nx_pydot import graphviz_layout # Alternative if pygraphviz not available

import warnings
warnings.filterwarnings('ignore')

<div style="background: rgb(255,165,0); border: solid 1px rgb(129,199,132); padding: 10px;">    

<h1>Section A: Sequence Overlaps and OLC</h1>

</div>

### Brief 

In Section A we will look at methods to compare sequences, and build an overlap graph. 

As biological sequence data is large, we need efficient ways to find shared regions between sequences. 
These methods apply to many situations, but here we will use them to build an overlap graph for OLC assembly, and to compare large sequences for regions of similarity. 

Question 1: Finding Overlaps <br>
Question 2: Building Overlap Graph <br>
Question 3: Minimizers for Genome-Genome alignment <br>

## Question 1: Identifying Read Overlaps

### Brief 

When performing OLC assembly, the first stage is to identify overlapping reads. 

This is a non-trivial task, as we need to compare each read to every other read for possible overlap. 

Given that in a typical long-read dataset we have 10,000 - 100,000 reads, and 1,000 - 100,000 bp read length, the computational time can very high without heuristics being used. 

For this reason, in question 1 we will first use **pairwise alignment** to find overlaps, then will add a **minhash heuristic** to speed up the process. 

<br>


For some parts in this question we will use reads from a .fastq file. The `A3.utils.read_fastq()` function loads reads from the fastq, where each read is only the sequence line. 

In [ ]:
from A3.utils import read_fastq

# load reads and print
reads = read_fastq(FASTQ2_PATH)
for i, read in enumerate(reads[:5]):
    print(f'read {i}: {read[:20]}...')

For this question a helper function, `calc_overlap_sg()`, has been provided to perform semi-global alignment of two sequences. 

- `calc_overlap_sg()` accepts two reads (str) as input, and optionally `should_print=True` to see the alignment.  
- `calc_overlap_sg()` returns a tuple of `(aln_score, read1_span, read2_span)` when there is an overlap.
- `calc_overlap_sg()` returns `None` when there is no overlap. 

This function will be used throughout question 1 for any pairwise alignments needed. 

Run the cell below to see examples of its use.


In [ ]:
from A3.utils import calc_overlap_sg

# demonstrating the calc_overlap_sg function
print('\nHELLOSUNDAY vs SATURDAYHELLO\n')
alignment = calc_overlap_sg(read1='HELLOSUNDAY', read2='SATURDAYHELLO', should_print=True)
print(f'output: {alignment}\n')

print('\nSUNDAYHELLOSUNDAY vs HELLO\n')
alignment = calc_overlap_sg(read1='SUNDAYHELLOSUNDAY', read2='H#LLO', should_print=True)
print(f'output: {alignment}\n')

print('\nKITTEN vs SITTING\n')
alignment = calc_overlap_sg(read1='KITTEN', read2='SITTING', should_print=True)
print(f'output: {alignment}\n')

print('\nKITTEN vs SUNDAYHELLOSUNDAY\n')
alignment = calc_overlap_sg(read1='KITTEN', read2='SUNDAYHELLOSUNDAY', should_print=True)
print(f'output: {alignment}\n')


<div style="color: rgb(27,94,32); background: rgb(200,230,201); border: solid 1px rgb(129,199,132); padding: 10px;">
<h3>Question 1a</h3>
    
<b>Challenge:</b> Write a function that determines whether there is an overlap between two reads. 
   
- [ ] Input: `read1: str`; `read2: str`; `min_score: int`; `min_overlap: int`; 
- [ ] Output: True or False (bool)
    
Notes: 
    
- [ ] Only return True if the alignment score >= min_score, and overlap len >= min_overlap.
- [ ] The alignment score and overlapping intervals for the two reads are provided by the `calc_overlap_sg()` function (see above)
     
</div>

In [ ]:
# -- GRADED CELL (2 marks) - complete this cell --

def has_overlap(read1, read2, min_score, min_overlap):
    """
    Returns True if there is an overlap between the two reads, else False. 
    Uses the calc_overlap_sg() function to do semi-global alignment. 
    """
    from A3.utils import calc_overlap_sg
    
    # YOUR CODE HERE
    # Extract the alignment score between 2 reads
    calc = calc_overlap_sg(read1, read2)

    # If there is an alignment proceed to the next step
    if calc != None:

        # Extract the alignment and overlap scores from the output
        score = calc[0]
        overlap = calc[1][1] - calc[1][0] 

        # If satisfies the conditions then it returns True
        if score >= min_score and overlap >= min_overlap:
            return True

        # Otherwise it's false
        else:
            return False
            
    else:
        return False

In [ ]:
# extra code cell for development if needed


In [ ]:
# Testing cell - Do not alter.

# Visible tests

# tests for simple strings
print(f"test1: expected=True, actual={has_overlap('SATURDAYHELLO', 'HELLOSUNDAY', 5, 5)}")
print(f"test2: expected=True, actual={has_overlap('SATURDAYHELLOSUNDAY', 'HELLO', 5, 5)}")
print(f"test3: expected=False, actual={has_overlap('SATURDAYHELLO', 'H#LLOSUNDAY', 5, 5)}")
print(f"test4: expected=False, actual={has_overlap('SATURDAYHELLO', 'H#LLOSUNDAY', 4, 5)}")
print(f"test5: expected=False, actual={has_overlap('SATURDAYHELLOSUNDAY', 'HELLO', 5, 10)}")
print(f"test6: expected=True, actual={has_overlap('KITTEN', 'SITTING', 1, 5)}")
print(f"test7: expected=False, actual={has_overlap('KITTEN', 'SITTING', 3, 5)}")

# tests for reads in readset
from A3.utils import read_fastq
reads = read_fastq(FASTQ1_PATH)
print(f"test8: expected=True, actual={has_overlap(reads[0], reads[1], 30, 100)}")
print(f"test9: expected=False, actual={has_overlap(reads[0], reads[5], 30, 100)}")


<div style="color: rgb(27,94,32); background: rgb(200,230,201); border: solid 1px rgb(129,199,132); padding: 10px;">
<h3>Question 1b</h3>
    
<b>Challenge:</b> Write a function to find all read overlaps for a given input readset. 
   
- [ ] Input: `reads: list[str]`; `min_score: int`; `min_overlap: int`;
- [ ] Output: overlaps as list of tuples, where each tuple has the format (read1, read2, read1_span, read2_span).

Notes:
    
- [ ] You can use the previous `has_overlap()` function from q1a to identify if two reads overlap if you like
- [ ] Only return overlaps with alignment score >= min_score, and overlap len >= min_overlap.
- [ ] When comparing (read1, read2) for overlap, do not also compare and return (read2, read1). 

The span of each overlap has the format [start, end]. For example, read1_span = [0, 5].<br>
This information is provided to you by the `calc_overlap_sg()` function. 
 

    
    
</div>

In [ ]:
# -- GRADED CELL (2 marks) - complete this cell --

def find_overlaps_sg(reads, min_score, min_overlap):
    """
    finds all pairwise overlaps between reads in a fastq file. 
    returns list of tuples, where each tuple consists of (read1, read2, read1_span, read2_span).
    only overlaps with an alignment score of >min_score are returned.
    """
    # YOUR CODE HERE
    # Create a list of output pairwise overlaps
    list_tup = []

    # Create a pointer
    i = 0

    # Do the while loop for all pairwise comparisons
    while i <= len(reads) - 2:

        # Storing the first read
        read1 = reads[i]

        # Compare read1 with all other possible reads through iterating read2
        for read in range(len(reads)-1-i):
            read2 = reads[read+1+i]

            # Extract the alignment score between 2 reads
            calc = calc_overlap_sg(read1, read2)

            # If there is an alignment proceed to the next step
            if calc != None:

                # Extract the alignment and overlap scores from the output
                score = calc[0]
                overlap = calc[1][1] - calc[1][0] 

                # If satisfies all conditions, extract the read1 and read2 alignment span from calc_overlap_sg function
                # Put it into a tup with (read1, read2, read1_span, read2_span) format
                # Store it into the list_tup
                if score >= min_score and overlap >= min_overlap:
                    tup = (read1, read2, calc[1], calc[2])
                    list_tup.append(tup)

        # Add the pointer to change read1 with the next read
        i += 1

    # Return the output in the list_tup
    return list_tup

In [ ]:
# extra code cell for development if needed
                

In [ ]:
# Testing cell - Do not alter.

# Visible tests

# --- test 1 ---
reads = ['HELLOSUNDAY', 'SATUDAYHELLO', 'THERE', 'SATURDAYTHERESUNDAY']

print('\n--- Test 1 ---')
print('\nexpected:')
print("""\
('HELLOSUNDAY', 'SATUDAYHELLO', (0, 5), (7, 12))
('THERE', 'SATURDAYTHERESUNDAY', (0, 5), (8, 13))\
""")

overlaps = find_overlaps_sg(reads, min_score=5, min_overlap=5)
print('\nactual:')
for ov in overlaps:
    print(ov)

# --- test 2 ---
reads = ['KITTEN', 'SITTING', 'CAKE', 'BLAKE']

print('\n--- Test 2 ---')
print('\nexpected:')
print("""\
('KITTEN', 'SITTING', (0, 6), (0, 6))\
""")

overlaps = find_overlaps_sg(reads, min_score=1, min_overlap=5)
print('\nactual:')
for ov in overlaps:
    print(ov)

# --- test 3 ---
from A3.utils import read_fastq
reads = read_fastq(FASTQ1_PATH)
print('\n--- Test 3: first 5 overlaps for readset of 33 long reads with avg length 1000 ---')
print('\nexpected:')
print("""\
(172, 798) (0, 627)
(604, 1021) (0, 417)
(433, 627) (0, 199)
(457, 924) (0, 477)
(424, 830) (0, 403)\
""")
print('\nactual:')
overlaps = find_overlaps_sg(reads, min_score=30, min_overlap=100)
for ov in overlaps[:5]:
    print(ov[2], ov[3])


<div style="color: rgb(27,94,32); background: rgb(200,230,201); border: solid 1px rgb(129,199,132); padding: 10px;">
<h3>Question 1c</h3>
 
You may have noticed that Test 3 for Question 1b ran for around 5-10 seconds. <br>
There are 33 reads in this readset, with 1000bp average length. <br>
Since the runtime of this test was a few seconds, a real long-read dataset would take days or weeks to complete! 
    
Unfortunately, on real data, finding overlaps via semi-global alignment is extremely slow. 
 
To remedy this, we will implement a minhash preprocessing step to identify reads which may have overlap before doing any alignment.   
       
<b>Challenge:</b> Write a function to create a minhash sketch for a given sequence. 
  
- [ ] Input: `sequence: str`; `k: int`; `n: int`;
- [ ] Output: minhash sketch of sequence (set of integers)   

Notes:

- [ ] Use the value of `k` for kmer size    
- [ ] Use `mmh3.hash(kmer, seed=SEED)` to calculate hash values
- [ ] Only return the `n` smallest kmer hash values

    

    
  

    
    
</div>

In [ ]:
# -- GRADED CELL (1 marks) - complete this cell --

def get_sketch(sequence, k, n):
    """
    calculates the minhash sketch for a sequence.
    returns a set of n integers.
    """
    import mmh3
    SEED = 42

    # YOUR CODE HERE
    # Create a list to store kmers
    kmers = []

    # Extract the kmers from the given sequence with k size
    # Use mmh3.hash(kmer, seed=SEED) to calculate hash values
    # Store it into the kmers list
    for i in range(len(sequence)-k+1):
        kmer = sequence[i:i+k]
        kmers.append(mmh3.hash(kmer, seed=SEED))

    # Sort the list with kmers inside
    kmers.sort()

    # Return the n smallest kmer hash values (set of integers)
    minhash = set(kmers[:n])

    return minhash

In [ ]:
# extra code cell for development if needed


In [ ]:
# Testing cell - Do not alter.

# Visible tests

# test 1: return type
print('\n--- test 1 - checking output type ---')
sketch = get_sketch('HELLO_THERE_FRIEND!', k=5, n=3)
print(f"expected=<class 'set'>, actual={type(sketch)}")

# test 2: toy example
print('\n--- test 2 - toy example ---')
sketch = list(get_sketch('HELLO_THERE_FRIEND!', k=5, n=3))
sketch.sort()
print('expected: [-1497645356, -1462843622, -1050684777]')
print(f"actual:   {sketch}")

# test 3: actual read seq
reads = read_fastq(FASTQ2_PATH)
seq = reads[0]
sketch = list(get_sketch(seq, k=8, n=5))
sketch.sort()
print('\n--- test 3 ---')
print('expected: [-2146380568, -2145247518, -2145038097, -2142884265, -2142628213]')
print(f"actual:   {sketch}")



<div style="color: rgb(27,94,32); background: rgb(200,230,201); border: solid 1px rgb(129,199,132); padding: 10px;">
<h3>Question 1d</h3>
  
<b>Challenge:</b> Write a function to calculate the jaccard index for two sets
   
- [ ] Input: `set1: set[str]`; `set2: set[str]`;
- [ ] Output: jaccard index as float

For this question, do not create a union sketch. Return the jaccard index according to the following formula: 
    
<img style="max-width: 300px" src="https://github.com/melbournebioinformatics/COMP90014/raw/A3/media/q1d_figure.PNG" />
    
<br>    

In [ ]:
# -- GRADED CELL (1 marks) - complete this cell --

def calc_jaccard(set1, set2):
    """
    calculates the jaccard index between two sets.
    returns a float.
    """
    # YOUR CODE HERE
    # Create jaccard formula given 2 sets of minhash values
    jaccard = abs(len(set1 & set2)) / (abs(len(set1)) + abs(len(set2)) - abs(len(set1 & set2)))

    # Return the jaccard index
    return jaccard


In [ ]:
# extra code cell for development if needed


In [ ]:
# Testing cell - Do not alter.

reads = read_fastq(FASTQ2_PATH)

# --- test 1: two overlapping reads --- 
sketch1 = get_sketch(reads[0], k=8, n=200)
sketch2 = get_sketch(reads[1], k=8, n=200)
print('\n--- test 1 (overlapping reads) ---')
print(f'expected=0.25, actual={calc_jaccard(sketch1, sketch2):0.2f}')

# --- test 2: two non-overlapping reads --- 
sketch1 = get_sketch(reads[0], k=8, n=200)
sketch2 = get_sketch(reads[4], k=8, n=200)
print('\n--- test 2 (non-overlapping reads) ---')
print(f'expected=0.02, actual={calc_jaccard(sketch1, sketch2):0.2f}')



<div style="color: rgb(27,94,32); background: rgb(200,230,201); border: solid 1px rgb(129,199,132); padding: 10px;">
<h3>Question 1e</h3>
  
<b>Challenge:</b> Write a single function to find all read overlaps for a given input readset, using a minhash preprocessing step as heuristic. 
      
- [ ] Input: `reads: list[str]`; `k: int`; `n: int`; `min_jaccard: float`; `min_score: int`; `min_overlap: int`;
- [ ] Output: `overlaps: list` a list of tuples, where each tuple has the format (read1_idx, read2_idx, read1_span, read2_span).
    
As our sequences are long, rather than returning the actual reads, return the index of each read in the overlap. That is:

- [ ] read1_idx is the index of read1 in the readset
- [ ] read2_idx is the index of read2 in the readset

Your function should have 2 steps. 
    
- Step 1: prescreen pairs of reads for possible overlap using `get_sketch()` and `calc_jaccard()`
- Step 2: for candidate pairs, perform semi-global alignment using `A3.utils.calc_overlap_sg()`
    
Notes:
- [ ] Step 1: only consider read pairs if jaccard index is >= `min_jaccard`
- [ ] Step 2: only return overlaps with alignment score >= `min_score`, and overlap len >= `min_overlap`.
- [ ] The `k` and `n` parameters should be passed to your `get_sketch()` function from q1c
- [ ] The test case below takes approximately 10 seconds to run on our test machine. 
- [ ] If your function takes more than 1 minute to run on our test machine, you will lose 1 mark.  
- [ ] The input readset has been ordered by position, so consecutive reads are likely to overlap.     
    


In [ ]:
# -- GRADED CELL (3 marks) - complete this cell --

def find_overlaps_minhash(reads, k, n, min_jaccard, min_score, min_overlap):
    """
    Finds all pairwise overlaps between reads in an input readset.
    - Has a minhash preprocessing step to identify possibly overlapping reads. 
    - Has a semi-global alignment step to validate alignment.
    
    Returns list of tuples, where each tuple consists of (read1, read2, read1_span, read2_span).
    """
    from A3.utils import calc_overlap_sg
    
    # YOUR CODE HERE
    # Create a list to store the output
    list_tup = []

    # Do for loop for all pairwise comparisons
    for i in range(len(reads)-1):

        # Minhashing read1 using previous get_sketch
        # Stored as set1
        set1 = get_sketch(reads[i], k, n)

        # Compare read1 with all other possible reads through iterating read2
        for j in range(i+1,len(reads)):
            
            # Minhashing read1 using previous get_sketch
            # Stored as set1
            set2 = get_sketch(reads[j], k, n)

            # Extract the jaccard index between the 2 sets
            jaccard = calc_jaccard(set1, set2)

            # If jaccard index pass the min index then proceed to the next step
            if jaccard >= min_jaccard:

                # Extract the alignment score between the 2 reads
                calc = calc_overlap_sg(reads[i], reads[j])

                # If there is an alignment proceed to the next step
                if calc != None:

                    # Extract the alignment and overlap scores from the output
                    score = calc[0]
                    overlap = calc[1][1] - calc[1][0] 

                    # If the alignment and overlap scores exceed the min scores then proceed to the next step
                    if score >= min_score and overlap >= min_overlap:

                        # Store it in the tuple of (read1_idx, read2_idx, read1_span, read2_span) format
                        # Append it into the list
                        tup = (i, j, calc[1], calc[2])
                        list_tup.append(tup)

                # If the alignment doesn't exist proceed to the next read
                else:
                    continue

            # If the jaccard index below the min index then proceed to the next read
            else:
                continue

    # Return the output list
    return list_tup

In [ ]:
# extra code cell for development if needed


In [ ]:
# Testing cell - Do not alter.

# Visible tests

print('\n--- Single Test ---')
print('expected:')
print("""\
i: 0	j: 1	span1: (2248, 5125)	span2: (0, 2858)
i: 1	j: 2	span1: (2540, 4994)	span2: (0, 2448)
i: 2	j: 3	span1: (1222, 4326)	span2: (0, 3098)
i: 3	j: 5	span1: (2582, 5320)	span2: (0, 2758)
i: 3	j: 6	span1: (3453, 5320)	span2: (0, 1863)
i: 5	j: 6	span1: (874, 3830)	span2: (0, 2939)
i: 5	j: 7	span1: (2493, 4649)	span2: (0, 2166)
i: 7	j: 8	span1: (2567, 4702)	span2: (0, 2142)
i: 7	j: 9	span1: (2896, 4702)	span2: (0, 1805)
i: 8	j: 9	span1: (332, 5131)	span2: (0, 4791)\
""")

print('\nactual:')
reads = read_fastq(FASTQ2_PATH)[:20]
overlaps = find_overlaps_minhash(reads, k=9, n=250, min_jaccard=0.1, min_score=30, min_overlap=100)
for read1_idx, read2_idx, read1_span, read2_span in overlaps[:10]:
    print(f'i: {read1_idx}\tj: {read2_idx}\tspan1: {read1_span}\tspan2: {read2_span}')



### Question 1f (short answer)

(2 marks, max 200 words)

<div class="alert alert-info">

While highly efficient, Minhash is a **heuristic** for finding read overlaps. 
- In what situations could the minhash heuristic not identify reads which genuinely have overlap? (1)
- In what manner does minhash sample kmers from a sequence? (0.5) 
- Briefly descibe another approach which samples kmers in a manner more suited to finding overlaps. (0.5)
    
</div>


<span style="color:rgb(17, 122, 121); font-family:Courier"><i><b># -- GRADED CELL (2 marks) - complete this cell --</b></i></span>

YOUR ANSWER HERE



- 1. When the reads are very short then there is a higher chance that the hash values will be the same even though the reads don't overlap.
  2. When there is a lot of repetitive sequences
  3. Sequencing errors that will change the hash values and cause it not to overlap.
- Minhash sample kmers using a random hash function to generate a hash value for each kmer. The minimum hash value for all kmers in the sequence is then selected. The minimum hash values are then stored.
- Minimizers and window. Using minimizers we don't need to extract every kmers, just partition them into separate windows and select the minimum kmer from that every windows according to an order.

## Question 2: Building Overlap Graph 

### Brief

Once read overlaps have been identified, we need to create an overlap graph from the overlaps. 

In question 2 we will identify the **relation** of each overlap, and will **build** our graph accordingly. 

<br>

<div style="color: rgb(27,94,32); background: rgb(200,230,201); border: solid 1px rgb(129,199,132); padding: 10px;">
<h3>Question 2a</h3>
  
<b>Challenge:</b> Write a function to determine the relationship of an overlap.
         
- [ ] Input: `read1: str`; `read2: str`; `read1_span: [int, int]`; `read1_span: [int, int]`;
- [ ] Output: The relationship [str] - either "contained", "contains", "suffix", or "prefix"

Notes:
    
- [ ] Consider the sequence lengths and the spans when identifying the relationship. 
- [ ] The frame of reference is from read1. See the following examples, and the test cases for more clarification. 
- [ ] In the test cases, read1 is referred to as "target" and read2 is referred to as "query"

<div style="width: 400px"><pre>
read2 is a 'suffix' relative to read1.

    read1: HELLO------
              ||
    read2: ---LOCATION
    
read2 is 'contained' relative to read1.
    
    read1: SUNDAYHELLOSUNDAY
                 |||||                  
    read2: ------HELLO------ 
</pre></div>

    
 <br>

In [ ]:
# -- GRADED CELL (2 marks) - complete this cell --

def get_relationship(read1, read2, read1_span, read2_span):
    """
    Determines the relationship between two reads.
    Returns a string of either 'contained', 'contains', 'suffix', or 'prefix'.
    
    Frame of reference is from read1. 
    """
    
    # YOUR CODE HERE
    # When read1 span starts from index 0 (= 1st character in a word) and read2 span doesn't start from index 0
    # 2 possibilities:
    # -Prefix if the length of read1 is longer than the overlap => the overlap in read2 is the prefix of read1
    # -Contains if the length of read1 is only the overlap => read2 contains read1
    if read1_span[0] == 0 and read2_span[0] != 0:
        relship = "prefix"
        
        if read1_span[1] == len(read1):
            relship = 'contains'  

    # Otherwise, it is the other way around, read2 starts from index 0 and read1 doesn't
    # 2 possibilities:
    # -Suffix if the length of read2 is longer than the overlap => the overlap in read2 is the suffix of read1
    # -Contained if the length of read2 is only the overlap => read2 is contained in read1
    elif read2_span[0] == 0 and read1_span[0] != 0:
        relship = "suffix"
        
        if read2_span[1] == len(read2):
            relship = "contained"

    # Return the relationship
    return relship

In [ ]:
# extra code cell for development if needed


In [ ]:
# Testing cell - Do not alter.

# Visible tests

print('\n--- test 1 ---\n')
read1 = 'HELLOSUNDAY'
read2 = 'SATURDAYHELLO'
_, read1_span, read2_span = calc_overlap_sg(read1, read2, should_print=True)
print(f'expected=prefix, actual={get_relationship(read1, read2, read1_span, read2_span)}')

print('\n--- test 2 ---\n')
read1, read2 = read2, read1
_, read1_span, read2_span = calc_overlap_sg(read1, read2, should_print=True)
print(f'expected=suffix, actual={get_relationship(read1, read2, read1_span, read2_span)}')

print('\n--- test 3 ---\n')
read1 = 'SUNDAYHELLOSUNDAY'
read2 = 'HELLO'
_, read1_span, read2_span = calc_overlap_sg(read1, read2, should_print=True)
print(f'expected=contained, actual={get_relationship(read1, read2, read1_span, read2_span)}')

print('\n--- test 4 ---\n')
read1, read2 = read2, read1
_, read1_span, read2_span = calc_overlap_sg(read1, read2, should_print=True)
print(f'expected=contains, actual={get_relationship(read1, read2, read1_span, read2_span)}')


<div style="color: rgb(27,94,32); background: rgb(200,230,201); border: solid 1px rgb(129,199,132); padding: 10px;">
<h3>Question 2b</h3>
  
<b>Challenge:</b> Write a function to create an overlap graph as a networkx.DiGraph from a list of reads.
         
- [ ] Input: `reads: list[str]`; `min_overlap: int`; `min_score: int`;
- [ ] Output: overlap graph as networkx.DiGraph
- [ ] Nodes in the graph should be reads
- [ ] Edges in the graph should be directed, indicating the overlap relationship, and be labelled with the overlap length. 
    
Notes:
- [ ] For each read, use the `calc_overlap_sg()` function to identify whether it overlaps any other read.
- [ ] If two reads overlap, use your previous `get_relationship()` function to identify the overlap relationship.
- [ ] **Only** add edges to the graph if the relationship is either "prefix" or "suffix".
- [ ] Edges should have a 'label' property with the length of the overlap.     
    
The test cell output should produce plots similar to the following:
    
<img src="https://github.com/melbournebioinformatics/COMP90014/raw/A3/media/q2b_figure.PNG" />

In [ ]:
# -- GRADED CELL (3 marks) - complete this cell --

def create_overlap_graph(reads, min_overlap=4, min_score=4):
    """
    Creates an overlap graph for a given set of reads.
    Uses the calc_overlap_sg() function to identify read overlaps. 
    Uses the get_relationship() function to determine the overlap relationship. 
    Only adds edges
    """
    from A3.utils import calc_overlap_sg
    graph = nx.DiGraph()
    
    # YOUR CODE HERE
    # Do for loop for all pairwise comparisons
    for i in range(len(reads)-1):
        read1 = reads[i]

        # Compare read1 with all other possible reads through iterating read2
        for j in range(i+1,len(reads)):
            read2 = reads[j]

            # Extract the alignment score between the 2 reads
            calc = calc_overlap_sg(read1, read2)

            # If there is an alignment proceed to the next step
            if calc != None:

                # Extract read1 and read2 overlap span from the output
                read1_span = calc[1]
                read2_span = calc[2]

                # Extract the alignment and overlap scores from the output
                score = calc[0]
                overlap = calc[1][1] - calc[1][0] 

                # If the alignment and overlap scores exceed the min scores then proceed to the next step
                if score >= min_score and overlap >= min_overlap:

                    # Extract the relationship from those 2 reads using previous get_relationship function
                    rel_ship = get_relationship(read1, read2, read1_span, read2_span)

                    # If read2 is prefix of read1
                    # Modify the graph by adding an edge from read2 to read1 with label the length of the overlap
                    if rel_ship == 'prefix':
                        graph.add_edge(read2, read1, label = overlap)

                    # Otherwise, read2 is suffix of read1
                    # Modify the graph by adding an edge from read1 to read2 with label the length of the overlap
                    elif rel_ship == 'suffix':
                        graph.add_edge(read1, read2, label = overlap)

            # If there is no alignment then continue to the next read
            else:
                continue

    # Return the modified graph
    return graph 

In [ ]:
# extra code cell for development if needed


In [ ]:
# Testing cell - Do not alter.
from A3.utils import draw_overlap_graph

# Visible tests
genome = "to_every_thing_thing_there_is_a_season"
reads = [
    'to_every_',
    'o_every_t',
    'ery_thing',
    '_thing_th',
    'ng_there_',
    'there_is_',
    'e_is_a_se',
    '_is_a_sea',
    '_a_season',
]

# test1: create graph from first two reads, check nodes and edges
print('\nTest 1: Checking graph nodes & edges')
graph = create_overlap_graph(reads[:2])
expected_nodes = ["to_every_", "o_every_t"]
expected_edges = [("to_every_", "o_every_t")]
expected_labels = [("to_every_", "o_every_t", {'label': 8})]
print(f'Nodes are correct? {True if list(graph.nodes) == expected_nodes else False}')
print(f'Edge directions are correct? {True if list(graph.edges) == expected_edges else False}')
print(f'Edge labels are correct? {True if list(graph.edges(data=True)) == expected_labels else False}')

# test2: create graph from first two reads
print('\nTest 2: Create graph from first two reads')
graph = create_overlap_graph(reads[:2])
draw_overlap_graph(graph)

# test3: create graph from first four reads
print('\nTest 3: Create graph from first four reads')
graph = create_overlap_graph(reads[:4])
draw_overlap_graph(graph)

# test4: create graph from all reads
print('\nTest 4: Create graph from all reads')
graph = create_overlap_graph(reads)
draw_overlap_graph(graph)



### Question 2C (short answer)

(2 marks, max 200 words)

<div class="alert alert-info">

What are 2 methods we can use to simplify an overlap graph? (1)

Why is OLC a more appropriate method than de bruijn graphs for assembling long-read sequencing data? (1)
    
</div>


<span style="color:rgb(17, 122, 121); font-family:Courier"><i><b># -- GRADED CELL (2 marks) - complete this cell --</b></i></span>

YOUR ANSWER HERE



- 2 methods to simplify an overlap graph:
  1. Transitive Edge Reduction
  2. Dead-End Removal

- 1. Because OLC is better at handling overlaps between long reads. Since de Bruijn graphs are divided into kmers, long reads with overlaps can have some kmers in common. This can make de Bruijn graph assemblers to distinguish between different reads into the correct order. 
  2. Better at handling sequencing errors since OLC uses a consensus algoritm that takes into account each read quality.
  3. More efficient in assembling long reads because OLC doesn't need to build a large graph of all the kmers in the reads. Less time-consuming and memory-intensive. 

<div style="background: rgb(255,165,0); border: solid 1px rgb(129,199,132); padding: 10px;">    

<h1>Section B: De Bruijn Assembly</h1>

</div>

### Brief

Sequencing errors can result in erroneous kmers which complicate De Bruijn graphs. If we do not account for these errors our assembly may become fragmented and contain spurious contigs. 

One method of reducing the impact of sequencing errors on our final assembly is to remove low-abundance kmers from the graph. In this section you will build a De Bruijn graph from a set of reads, simplify the graph by removing erroneous kmers, and coalesce linear chains of nodes. 


<div style="color: rgb(27,94,32); background: rgb(200,230,201); border: solid 1px rgb(129,199,132); padding: 10px;">
<h3>Question 3a</h3>
    
<b>Challenge:</b> Write a function to extract kmers from a set of sequencing reads and count their abundances. Your function should return a dictionary where keys are kmers and values are kmer counts.
   
- [ ] Input: `dna_strings: list[str]`; `k: int`
- [ ] Handle cases where k is > read length
- [ ] Ouput: dict with kmers as keys and counts as values

</div>

In [ ]:
# -- GRADED CELL (1 marks) - complete this cell --

def count_kmers(dna_strings, k):
    """
    Count k-mers in a list of DNA strings and return a dictionary of k-mer counts.

    Args:
    dna_strings (list of str): A list of DNA strings for k-mer counting.
    k (int): The length of k-mers to count.

    Returns:
    dict: A dictionary where keys are k-mers and values are their counts in the input DNA strings.
    """
    # optional, you may use defaultdict if you like
    from collections import defaultdict
    
    # YOUR CODE HERE
    # Create a default dictionary
    my_dict = defaultdict(int)

    # Iterating through the list of dna_strings
    for i in range(len(dna_strings)):

        # Add a pointer
        kmer = 0

        # Slicing the dna_strings into k size (kmer) and adding it into the dictionary so long satisfies the conditions
        # Handle cases where k is > read length
        while kmer < len(dna_strings[i])-k+1 and k < len(dna_strings[i]):
            my_dict[dna_strings[i][kmer:kmer+k]] += 1
            
            # Adding the pointer to shift the kmer
            kmer += 1

    # Return a dictionary where keys are kmers and values are kmer counts.
    return my_dict

In [ ]:
# extra code cell for development if needed


In [ ]:
# Testing cell - Do not alter.

# Visible tests
reads = ['CATTGGC', 'ATTGGCC', 'TTGGCCA', 'TGGCCAA', 'CATCGGCCT']
kmer_counts = count_kmers(reads, k=4)

# single test 
print('\n--- Single Test ---')
print('\nexpected')
print("""\
CATT 1
ATTG 2
TTGG 3
TGGC 4
GGCC 4
GCCA 2
CCAA 1
CATC 1
ATCG 1
TCGG 1
CGGC 1
GCCT 1\
""")
print('\nactual:')
for k, v in kmer_counts.items():
    print(k, v)


<div style="color: rgb(27,94,32); background: rgb(200,230,201); border: solid 1px rgb(129,199,132); padding: 10px;">
<h3>Question 3b</h3>
    
<b>Challenge:</b> Write a function to build a De Bruijn graph from a set of reads for a given kmer length. 

The graph should be created as a networkx DiGraph, storing the kmer name as edge 'label'.
   
- [ ] Input: `reads: list[str]`; `k: int`
- [ ] Ouput: de bruijn graph as nx.DiGraph
    
Notes:

- [ ] For simplicity, treat all reads as originating from the +ve strand. 

</div>

In [ ]:
# -- GRADED CELL (1 marks) - complete this cell --

def build_debruijn_graph(reads, k):
    '''
    Given a set of reads and a value k, return a de Bruijn graph as a nx.DiGraph.
    '''
    graph = nx.DiGraph()

    # YOUR CODE HERE
    # Creating a dictionary of kmer counts by calling previous count_kmers function
    my_dict = count_kmers(reads, k)

    # Create a list of kmers
    keys = []
    for key in my_dict:
        keys.append(key)

    # Building de Bruijn graph by splicing each kmer into prefix and suffix
    # Adding an edge from prefix -> suffix with kmer as its label
    for i in range(len(keys)):
        for j in range(len(keys[i])):
            prefix = keys[i][:-1]
            suffix = keys[i][1:]
            graph.add_edge(prefix, suffix, label= keys[i])

    # Return the modified graph
    return graph
    

In [ ]:
# extra code cell for development if needed


In [ ]:
# Testing cell - Do not alter.

# Visible tests

# --- single test ---
reads = ['CATTGGC', 'ATTGGCC', 'TTGGCCA', 'TGGCCAA', 'CATCGGCCT']
graph = build_debruijn_graph(reads, k=4)
nodes = list(graph.nodes)
nodes.sort()
edges = list(graph.edges(data=True))
edges.sort()

print('\n(Single Test)')
print('\n--- nodes ---')
print('\nexpected nodes:')
print("['ATC', 'ATT', 'CAA', 'CAT', 'CCA', 'CCT', 'CGG', 'GCC', 'GGC', 'TCG', 'TGG', 'TTG']")
print('\nactual nodes:')
print(nodes)

print('\n--- edges ---')
print('\nexpected edges:')
print("""\
ATC  --- ATCG --->  TCG
ATT  --- ATTG --->  TTG
CAT  --- CATC --->  ATC
CAT  --- CATT --->  ATT
CCA  --- CCAA --->  CAA
CGG  --- CGGC --->  GGC
GCC  --- GCCA --->  CCA
GCC  --- GCCT --->  CCT
GGC  --- GGCC --->  GCC
TCG  --- TCGG --->  CGG
TGG  --- TGGC --->  GGC
TTG  --- TTGG --->  TGG\
""")
print('\nactual edges:')
for n1, n2, data in edges:
    print(f'{n1}  --- {data["label"]} --->  {n2}')


<div style="color: rgb(27,94,32); background: rgb(200,230,201); border: solid 1px rgb(129,199,132); padding: 10px;">
<h3>Question 3c</h3>
    
<b>Challenge:</b> Write a function simplify the De Bruijn graph by removing nodes which have low support. 
   
- [ ] Input: `graph: nx.DiGraph`; `kmer_counts: dict`; `cutoff: int`
- [ ] Ouput: Simplified graph (nx.DiGraph)
    
Here, "support" is calculated by considering edges to and from a given node. <br>
Consider the maximum kmer frequecy for edges both to and from the node.

Example: In the image below, nodes 2, 3, 4 & 12 would be removed given `cutoff` is 10. 
    
<img display="block" src="https://github.com/melbournebioinformatics/COMP90014/raw/A3/media/q3c_figure.PNG" align="left" style="max-width: 600px"/>
    
</div>

In [ ]:
# -- GRADED CELL (3 marks) - complete this cell --

def remove_low_support_nodes(graph, kmer_counts, cutoff):
    '''
    Removes nodes from the graph if max support < cutoff. 
    Considers both the connections to and connections from nodes. 
    '''
    # YOUR CODE HERE
    # 
    nodes = list(graph.nodes())
    
    for i in range(len(nodes)):
        in_edge = list(graph.in_edges(nodes[i]))
        out_edge = list(graph.edges(nodes[i], data=True))
    
        max_edge_fr = 0
        max_edge_to = 0
        
        if len(in_edge) > 0:
            edge_fr = []
            
            for j in range(len(in_edge)):
                a = list(graph.edges(in_edge[j][0], data=True))
                b = [(in_, out_, kmer) for (in_, out_, kmer) in a if out_ == nodes[i]]
                kmer = kmer_counts[b[0][2]['label']]
                edge_fr.append(kmer)
            
            max_edge_fr = max(edge_fr)

        
        if len(out_edge) > 0:
            edge_to = []
            
            for k in range(len(out_edge)):
                kmer = out_edge[k][2]['label']
                edge_to.append(kmer_counts[kmer])
    
            max_edge_to = max(edge_to)

        
        if max(max_edge_fr, max_edge_to) < cutoff:
            graph.remove_node(nodes[i])
        
    return graph
    

In [ ]:
# extra code cell for development if needed


In [ ]:
# Testing cell - Do not alter.

graph = pickle.load(open(DBG1_FILEPATH, 'rb'))
kmer_counts = {
    "CATT": 3,
    "ATTG": 4,
    "TTGG": 9,
    "TGGC": 1,
    "CATC": 32,
    "ATCG": 44,
    "TCGG": 9,
    "CGGC": 27,
    "GGCC": 35,
    "GCCA": 15,
    "CCAA": 17,
    "GCCT": 8,
}

# visible tests

print('\n--- Single Test ---')
print('\nnodes before simplification')
print(list(graph.nodes))

print('\nnodes after simplification')
graph = remove_low_support_nodes(graph, kmer_counts, cutoff=10)
print(f'expected: [1, 8, 5, 6, 7, 9, 10, 11]')
print(f'actual:   {list(graph.nodes)}')


<div style="color: rgb(27,94,32); background: rgb(200,230,201); border: solid 1px rgb(129,199,132); padding: 10px;">
<h3>Question 3d</h3>
    
<b>Challenge:</b> Write a function simplify the De Bruijn graph by coalescing linear chains of nodes (with no branch points).

- [ ] Input: `graph: nx.DiGraph`
- [ ] Ouput: Simplified graph
    
For this question, you **do not** need to update a kmer_counts dict when coalescing nodes. 
 
Notes:
- [ ] Once a linear chain has been merged into a single node, ensure you regraft the relevant edges accordingly.
- [ ] Ensure you remove merged nodes from the graph. 
 
See the expected before and after in the following figure:

<img display="block" src="https://github.com/melbournebioinformatics/COMP90014/raw/A3/media/q3d_figure.PNG" align="left" style="max-width: 600px"/>
    
</div>

In [ ]:
# -- GRADED CELL (3 marks) - complete this cell --

def coalesce_linear_chains(graph):
    '''
    Simplifies a graph by coalescing linear chains of nodes. 
    Returns the simplified graph. 
    '''
    # YOUR CODE HERE
    # Create lists for dfs traversing and visited nodes
    visited = []
    stack = []

    # Looking for the starting node which has no in_edges, store that to stack
    for i in list(graph.nodes()):
        if len(graph.in_edges(i)) == 0:
            stack.append(i)

    # Create an outer while loop for traversing the nodes using dfs method (so long stack has nodes)
    # Pop node from stack and store it to current_node
    # Update the visited list
    while stack:
        current_node = stack.pop(0)
        visited.append(current_node)

        # Create an inner loop to merge the nodes that satisfy conditions in checkpoint
        while checkpoint(current_node, graph) == True:

            # Looking for the neighbor of the current node
            # Extracting the last character from neighbour node
            # Combining the current node with the last character from neighbour node
            edges = list(graph.neighbors(current_node))
            suffix = edges[0][-1]
            combine = f'{current_node + suffix}'

            # Extracting the predecessor of the current node and successor of the neighbor node
            predecessor = list(graph.predecessors(current_node))
            next_neighbour = list(graph.neighbors(edges[0]))

            # If the length of predecessor > 0
            # Add new edge from every predecessors with the new node (combined) and update the label accordingly
            if len(predecessor) != 0:
                for prev_edge in range(len(predecessor)):
                    graph.add_edge(predecessor[prev_edge], combine,
                                   label = graph.get_edge_data(predecessor[prev_edge], current_node)['label'])

            # If the length of successor > 0
            # Add new edge from new node (combined) with every successors and update the label accordingly
            if len(next_neighbour) != 0:
                for next_edge in range(len(next_neighbour)):
                    graph.add_edge(combine, next_neighbour[next_edge],
                                   label= graph.get_edge_data(edges[0], next_neighbour[next_edge])['label'])

            # Remove the individual nodes that are combined
            graph.remove_node(current_node)
            graph.remove_node(edges[0])

            # Now traversing to the combined node to see if we can merge with the next node again
            current_node = combine

        # If the checkpoint is False -> there is a branch so it doesnt satisy the conditions to be merged
        # Skip the node and traversing to the next neighbour nodes by storing them to the stack
        for j in list(graph.neighbors(current_node)):
            if j not in stack:
                stack.append(j)

    # Return the updated graph
    return graph 


# Make a function to check whether the current_node has 1 out_edge and the neighbor node has 1 in_edge
# If satisfies the condition it will return True and it will start to merge the nodes
# Otherwise it returns False and traversing to the next nodes
def checkpoint(current_node, graph):
    neighbour = list(graph.neighbors(current_node))

    if len(neighbour) == 1 and len(list(graph.in_edges(neighbour[0]))) == 1:
        return True
    else:
        return False
        

In [ ]:
# extra code cell for development if needed


In [ ]:
# Testing cell - Do not alter.

graph = pickle.load(open(DBG2_FILEPATH, 'rb'))

# Visible tests

print('\n(Single Test)')
graph = coalesce_linear_chains(graph)
nodes = list(graph.nodes)
nodes.sort()
edges = list(graph.edges(data=True))
edges.sort()
print('\n--- nodes after simplification ---')
print(f"expected: ['ATCGG', 'ATTGG', 'CAA', 'CAT', 'CCAA', 'CCT', 'GGCC']")
print(f'actual:   {nodes}')

print('\n--- edges after simplification ---')
print('\nexpected:')
print("""\
ATCGG   --- CGGC --->   GGCC
ATTGG   --- TGGC --->   GGCC
CAT     --- CATC --->   ATCGG
CAT     --- CATT --->   ATTGG
GGCC    --- GCCA --->   CCAA
GGCC    --- GCCT --->   CCT\
""")
print('\nactual:')
for n1, n2, data in edges:
    edge = f"--- {data['label']} --->"
    print(f'{n1:8s}{edge:^10s}   {n2}')



### Question 3e (short answer)

(3 marks, max 200 words)

<div class="alert alert-info">

We have simplified our de Bruijn graph by eliminating low coverage nodes with the assumption that these rare kmers are the result of sequencing errors. 
    
- How will this approach handle non-random errors? (1)
- Does this approach preserve all genuine DNA sequenced from the organism? (1)
- Can you suggest any biological explanation for observing lower than expected kmer abundance in a sequencing dataset? (1)
    
</div>


<span style="color:rgb(17, 122, 121); font-family:Courier"><i><b># -- GRADED CELL (3 marks) - complete this cell --</b></i></span>

YOUR ANSWER HERE



- It will remove some of the non-random errors however, non-random errors that have high coverage nodes won't be thrown simply because we only eliminate the non-random errors that have low coverage. One way to handle is to use a sequencing error correction algorithm before creating a de Bruijn graph.

- Eliminating low coverage nodes may make the graph seem simpler however it comes with consequences. Some genuine DNA sequences that may be present at low coverage due to multiple factors (sequencing errors, low expression levels, etc.) may be thrown out. Hence, eliminating low coverage nodes from de Bruijn graph may not preserve all genuine DNA sequenced from the organism

- 1. Sequencing errors can lead to kmers that are not present in the actual genome. These kmers will have lower than expected abundance in the sequencing dataset.
  2. Some genes and other DNA sequences are expressed at low levels in a cell.
  3. Some DNA sequences are rare in the population (rare variants).
  4. Allelic imbalance occurs when there is an unequal representation of the two alleles of a gene in a cell
  5. DNA damage can lead to changes in the DNA sequence. These changes can lead to kmers that are not present in the actual genome, or to kmers that are present at lower than expected abundance in the sequencing dataset.


### Question 3f (short answer)

(3 marks, max 200 words)

<div class="alert alert-info">

What topological features in de Bruijn graphs can be caused by repeats? (1)

How does our choice of kmer length affect the impact of repeats on our graph? (1)
    
What approaches can help resolve repetitive regions in the organism to produce better assemblies? (1)
    
</div>


<span style="color:rgb(17, 122, 121); font-family:Courier"><i><b># -- GRADED CELL (3 marks) - complete this cell --</b></i></span>

YOUR ANSWER HERE



- Bubbles, spurs, or erroneous edges
- A longer kmer size will make repeats less likely to create different kmers and capture more read span of DNA-sequence. High kmer size will have more specific kmers produced. Hence, it is better to use long kmer size when dealing with repeats
- A hybrid approach of short reads and long reads. Long reads has the advantage to span repetitive regions although it is less accurate compared to short-reads. However, we can fix that using short reads. 

<div style="background: rgb(255,165,0); border: solid 1px rgb(129,199,132); padding: 10px;">    

<h1>Section C: Genomic Intervals</h1>

</div>

Interval Search Trees (ISTs) are a data structure that can be used to efficiently query genomic annotations in a defined region. They are conceptually similar to Binary Search Trees which we have discussed in the tutorials. In this section you will query and modify an IST of gene spans, and will then build your own IST from a set of gene intervals. 

The IST appearing in Q4a-c was build from the following data: 

<div style="max-width: 300px">  
<pre>gene_name  start  end
gene_A     10     15
gene_B     20     30
gene_C     11     18
gene_D     13     20
gene_E     21     23
gene_F     22     32
gene_G     6      12
</pre></div>

Our IST has been structured so that node ids are the "start" field above, with the "gene_name" and "end" appearing as node attributes. 

For simplicity the trees in this question do not need to support multiple intervals with shared starting indices.

Run the cells below to view the tree, and the data for a particular node. 


In [ ]:
# visualisation helper function
def draw_interval_tree(graph, title):
    fig = plt.figure(1, figsize=(7, 7), dpi=60)
    if title is not None:
        plt.title(title, fontsize=14)
    pos = graphviz_layout(graph, prog="dot")
    nx.draw_networkx_nodes(graph, pos, node_color='white', node_size=1000, edgecolors='black', linewidths=2)
    nx.draw_networkx_edges(graph, pos, width=2)
    nx.draw_networkx_labels(graph, pos, font_size=12, font_family="sans-serif")
    nx.draw_networkx_edge_labels(
        graph, pos, font_color='red', font_size=14, 
        edge_labels={e: graph.edges[e]['label'] for e in graph.edges}
    )
    plt.tight_layout()
    plt.show()

In [ ]:
tree = pickle.load(open(IST1_FILEPATH, 'rb'))

# drawing our tree
draw_interval_tree(tree, 'Gene intervals IST')

# printing a node and its attributes
node = 20
data = tree.nodes[node]
print(f'node={node}, data={data}')

<div style="color: rgb(27,94,32); background: rgb(200,230,201); border: solid 1px rgb(129,199,132); padding: 10px;">
<h3>Question 4a</h3>
   
One of the ways ISTs provide efficient search time for intervals is by storing a "max_end" attribute at each node. 
The "max_end" attribute holds the maximum endpoint value among all intervals in the subtree rooted at that node.

This means that when searching the tree, we can use the "max_end" attribute at a given node to help us identify whether to keep traversing the tree for a matching interval, or if we can stop early. 
    
<b>Challenge:</b> Write a function to set the "max_end" attribute on each node in an IST.  
    
- [ ] Input: `tree: nx.DiGraph`; `root: int`;
- [ ] Output: updated tree (nx.DiGraph)
    
Notes:
- [ ] The `root` parameter will be used in question 4 to keep track of which node is the root.
    
</div>
    

In [ ]:
# -- GRADED CELL (2 marks) - complete this cell --

def set_max_ends(tree, root):
    
    # YOUR CODE HERE
    visited = list()
    
    dfs(visited, tree, root)

    return tree
    
def dfs(visited, tree, root):
    if root not in visited:
        visited.append(root)
        
        max_val = tree.nodes[root]['end']

        for neighbour in tree[root]:
            val = dfs(visited, tree, neighbour)

            # update max_val if needed
            max_val = max(max_val, val)
         
        tree.nodes[root]['max_end'] = max_val

    return max_val

In [ ]:
# extra code cell for development if needed


In [ ]:
# Testing cell - Do not alter.

# Visible tests

tree = pickle.load(open(IST1_FILEPATH, 'rb'))
root = 13
set_max_ends(tree, root)
print(f'test1 (node=13): expected=32, actual={tree.nodes[13]["max_end"]}')
print(f'test2 (node=6):  expected=12, actual={tree.nodes[6]["max_end"]}')
print(f'test3 (node=10): expected=18, actual={tree.nodes[10]["max_end"]}')


<div style="color: rgb(27,94,32); background: rgb(200,230,201); border: solid 1px rgb(129,199,132); padding: 10px;">
<h3>Question 4b</h3>

Now that we have added the "max_end" attribute for nodes, we can efficiently search the IST for intervals.     
    
<b>Challenge:</b> Write a function to get the "gene_name" of the first interval encountered, given an query range.
    
- [ ] Input: `tree: nx.DiGraph`; `root: int`; `query_start: int`; `query_end: int`
- [ ] Output: the "gene_name" attribute for the first valid interval encountered
- [ ] If there is no inverval in the tree overlapping the given query, `return None`
    
Notes:
- [ ] Traverse the tree from root
- [ ] Check if current node intersects with query
- [ ] If no left subtree, or "max_end" of left subtree is less than `query_start`, search right subtree
- [ ] Else search left subtree
    
</div>

In [ ]:
# helper function to get the left or right child of a node
def get_child(tree, parent, label):
    # get edges
    edges = tree.edges(parent, data=True)

    # no children
    if len(edges) == 0:
        return None
    
    # has children
    for edge in edges:
        if edge[2]['label'] == label: 
            return edge[1]  
    
    # has no children with correct label
    return None

In [ ]:
# -- GRADED CELL (2 marks) - complete this cell --

def get_gene(tree, root, query_start, query_end):
    '''
    For a given search range (query_start - query_end), returns the "gene_name" of the first valid interval encountered. 
    If no interval overlaps the search range, returns None. 
    '''
    # YOUR CODE HERE
    gene_name = ''
    
    if root != None:
        max_end = tree.nodes[root]['max_end']
        
        if root in range(query_start, query_end):
            gene_name = tree.nodes[root]['gene_name']
            
        elif query_end < max_end:
            root = get_child(tree, root, 'left')
            gene_name = get_gene(tree, root, query_start, query_end)
                
        elif query_end > max_end:
            root = get_child(tree, root, 'right')
            gene_name = get_gene(tree, root, query_start, query_end)

    else:
        gene_name = None
        
    return gene_name


In [ ]:
# extra code cell for development if needed


In [ ]:
# Testing cell - Do not alter.

tree = pickle.load(open(IST2_FILEPATH, 'rb'))
root = 13

# Visible tests
print(f'expected=gene_A, actual={get_gene(tree, root, 1, 11)}')
print(f'expected=gene_G, actual={get_gene(tree, root, 1, 7)}')
print(f'expected=None,   actual={get_gene(tree, root, 1, 2)}')


<div style="color: rgb(27,94,32); background: rgb(200,230,201); border: solid 1px rgb(129,199,132); padding: 10px;">
<h3>Question 4c</h3>

Depending on use case, we may want to add more interval data to the tree. <br>
One such example is when we want to combine multiple sets of interval data in a single tree. For example, combining read alignments with gene features to identify reads which are aligning to a particular gene region. 
    
For this question, we will just add another gene interval to the tree. 
    
<b>Challenge:</b> Write a function to add a new interval to the tree. 
    
- [ ] Input: `tree: nx.DiGraph`; `root: int`; `interval: tuple`
- [ ] Output: the updated tree (nx.DiGraph)  

Notes:
- [ ] Make sure to update the "max_end" attribute for upstream nodes in the tree if required.
    
</div>

In [ ]:
# -- GRADED CELL (2 marks) - complete this cell --

def add_interval(tree, root, interval):
    '''
    Adds an interval to an Interval Search Tree.
    Updates the "max_end" attribute for upstream nodes in the tree if required. 
    Returns the updated tree. 

    Args:
    tree (digraph): An interval search tree
    root (int): the root node of the tree
    interval (tuple): An interval as a tuple of (gene_name, start, end)
    '''
    # YOUR CODE HERE
    query_start = interval[1]
    query_end = interval[2]

    gene_name = ''
    if root != None:
        max_end = tree.nodes[root]['max_end']
        
        if root in range(query_start, query_end):
            print(tree.nodes[root]['gene_name'])
            
        elif query_end < max_end:
            root = get_child(tree, root, 'left')
            get_gene(tree, root, query_start, query_end)
                
        elif query_end > max_end:
            root = get_child(tree, root, 'right')
            gene_name = get_gene(tree, root, query_start, query_end)

    else:
        tree.add_edges()
        label = 'left' if number < node else 'right'
            tree.add_edge(node, number, label=label)
    
    
    
    return tree


In [ ]:
# extra code cell for development if needed
tree = pickle.load(open(IST2_FILEPATH, 'rb'))
tree.add_edge(6,7, label = 'left')

draw_interval_tree(tree, 'Gene intervals IST')

In [ ]:
# Testing cell - Do not alter.
tree = pickle.load(open(IST2_FILEPATH, 'rb'))
root = 13

# Visible tests

# new node added, node name correct
tree = add_interval(tree, root, ('gene_H',16,55))
print('\n--- test1: node id correct? ---')
print(f'expected=True, actual={True if 16 in tree.nodes else False}')

# new node parent correct
print('\n--- test2: parent of new node is correct? ---')
print(f'expected=20, actual={list(tree.predecessors(16))[0]}')

# new node attributes correct
print('\n--- test3: new node attributes are correct? ---')
print(f'expected: gene_name=gene_H, end=55, max_end=55')
node = tree.nodes[16]
print(f'actual:   gene_name={node["gene_name"]}, end={node["end"]}, max_end={node["max_end"]}')

# max_end attributes correct
print('\n--- test4: max_end attributes are correct? ---')
print('expected:')
print("""\
node=13, max_end=32
node=10, max_end=18
node=6, max_end=12
node=11, max_end=18
node=21, max_end=32
node=20, max_end=55
node=22, max_end=32
node=16, max_end=55\
""")
print('\nactual:')
for node in tree.nodes():
    print(f'node={node}, max_end={tree.nodes[node]["max_end"]}')
    


### Question 4d (short answer)

(2 marks, max 200 words)

<div class="alert alert-info">

How does the order in which items are added to a tree affect lookup time complexity? (2)
    
</div>


<span style="color:rgb(17, 122, 121); font-family:Courier"><i><b># -- GRADED CELL (2 marks) - complete this cell --</b></i></span>

YOUR ANSWER HERE



- It can affect the balancing properties of the tree. A tree that is balanced will have a lower lookup time complexity.
- If the order of the items added is decreasing then it will make a binary searh tree becomes linear tree which can increase the lookup time complexity from O(logn) to O(n)

<div style="background: rgb(255,165,0); border: solid 1px rgb(129,199,132); padding: 10px;">    
<h1>END OF ASSIGNMENT</h1>
</div>


## Submitting

Follow these steps to submit your assignment

1) Before you turn this assignment in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

2) Make sure you have filled in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE"

3) **Clear all output** (in the menubar, select Kernel$\rightarrow$Restart & Clear Output)

4) Your completed notebook file containing all your answers must be turned in via LMS in `.ipynb` format.

5) You must also submit a copy of this notebook in `html` format with the output cleared (see step 3).


Your submission should include **only two** files with names formatted as: **Assignment_3.ipynb** and **Assignment_3.html**